This notebook works through training and evaluating data provided by the NFL Big Data Bowl Kaggle Competition. Vector based feature representation models were trained and evaluated to predict which reciever a quarterback will throw to during a pass play.

#Load Packages

In [ ]:
!pip install pygam

In [6]:
import pandas
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

#import pygam

import itertools

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

import sklearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, mutual_info_score
from sklearn.model_selection import GridSearchCV, train_test_split

#Import Data and Train-Test Split

In [35]:
PlayFeatures = pd.read_csv("/content/drive/MyDrive/DS5500/Phase 2/PlayFeaturesForModels_12July23.csv")
PlayFeatures['trueTargetLabels'] = PlayFeatures.trueTargetLabels.astype('category')
PlayFeatures.head()

,qb_features_x,qb_features_y,qb_features_s,qb_features_a,qb_features_orient,qb_features_dir,target_0_x,target_0_y,target_0_s,target_0_a,target_0_o,target_0_dir,target_0_dir_orient_diff,target_0_qb_dist,target_0_qb_orient_diff,target_0_qb_dir_diff,target_0_closest_dist,target_0_closest_s,target_0_closest_a,target_0_closest_orient_diff,target_0_closest_dir_diff,target_0_closest_3_avg_dist,target_0_closest_3_max_s,target_0_closest_3_max_a,target_1_x,target_1_y,target_1_s,target_1_a,target_1_o,target_1_dir,target_1_dir_orient_diff,target_1_qb_dist,target_1_qb_orient_diff,target_1_qb_dir_diff,target_1_closest_dist,target_1_closest_s,target_1_closest_a,target_1_closest_orient_diff,target_1_closest_dir_diff,target_1_closest_3_avg_dist,target_1_closest_3_max_s,target_1_closest_3_max_a,target_2_x,target_2_y,target_2_s,target_2_a,target_2_o,target_2_dir,target_2_dir_orient_diff,target_2_qb_dist,target_2_qb_orient_diff,target_2_qb_dir_diff,target_2_closest_dist,target_2_closest_s,target_2_closest_a,target_2_closest_orient_diff,target_2_closest_dir_diff,target_2_closest_3_avg_dist,target_2_closest_3_max_s,target_2_closest_3_max_a,target_3_x,target_3_y,target_3_s,target_3_a,target_3_o,target_3_dir,target_3_dir_orient_diff,target_3_qb_dist,target_3_qb_orient_diff,target_3_qb_dir_diff,target_3_closest_dist,target_3_closest_s,target_3_closest_a,target_3_closest_orient_diff,target_3_closest_dir_diff,target_3_closest_3_avg_dist,target_3_closest_3_max_s,target_3_closest_3_max_a,target_4_x,target_4_y,target_4_s,target_4_a,target_4_o,target_4_dir,target_4_dir_orient_diff,target_4_qb_dist,target_4_qb_orient_diff,target_4_qb_dir_diff,target_4_closest_dist,target_4_closest_s,target_4_closest_a,target_4_closest_orient_diff,target_4_closest_dir_diff,target_4_closest_3_avg_dist,target_4_closest_3_max_s,target_4_closest_3_max_a,week,quarter,down,yardsToGo,yardlineNumber,defendersInTheBox,numberOfPassRushers,preSnapVisitorScore,preSnapHomeScore,absoluteYardlineNumber,trueTargetLabels,target_0_position_FB,target_0_position_RB,target_0_position_TE,target_0_position_WR,target_1_position_FB,target_1_position_RB,target_1_position_TE,target_1_position_WR,target_2_position_FB,target_2_position_RB,target_2_position_TE,target_2_position_WR,target_3_position_FB,target_3_position_RB,target_3_position_TE,target_3_position_WR,target_4_position_FB,target_4_position_RB,target_4_position_TE,target_4_position_WR,Time_Day,Time_Night
0,23.74,26.853333,1.44,4.25,37.54,75.83,24.92,27.383333,3.70,3.87,294.66,296.70,2.04,1.293561,-257.12,-220.87,9.935819,3.29,2.98,-233.58,25.76,10.179599,3.29,3.44,28.68,32.193333,6.05,0.72,231.62,215.54,16.08,7.274558,-194.08,-139.71,4.557477,2.76,2.29,-187.91,-7.07,7.688528,2.76,3.44,33.85,31.793333,5.43,1.41,267.41,270.55,3.14,11.252364,-229.87,-194.72,1.726297,2.76,2.29,-223.70,-62.08,3.809690,2.76,3.44,34.25,18.723333,5.79,2.84,265.07,262.10,2.97,13.287475,-227.53,-186.27,2.488453,3.72,3.45,-18.54,30.39,4.861545,3.72,3.45,37.02,8.903333,6.21,3.04,279.62,275.90,3.72,22.328477,-242.08,-200.07,3.931272,5.34,0.76,-89.62,4.95,8.427943,5.34,3.45,1,1,1,15,20,7,4,0,0,90,4,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1
1,66.48,23.773333,1.77,3.62,30.44,84.49,64.21,25.503333,2.75,2.27,246.15,181.30,64.85,2.854085,-215.71,-96.81,9.952336,2.88,1.51,-181.61,167.65,11.974993,3.97,2.55,71.13,14.433333,6.31,0.80,6.08,335.05,328.97,10.433509,24.36,-250.56,2.243769,3.97,2.14,75.46,21.90,6.393572,3.97,3.90,72.80,28.213333,4.76,3.75,302.54,324.44,21.90,7.723730,-272.10,-239.95,1.108873,2.88,1.51,-238.00,24.51,7.189092,3.97,2.55,73.01,15.973333,3.71,0.62,213.95,250.50,36.55,10.172556,-183.51,-166.01,1.455232,3.97,2.14,-132.41,106.45,4.902522,3.97,3.90,74.52,43.493333,6.17,1.67,281.86,268.85,13.01,21.296009,-251.42,-184.36,5.877772,3.14,1.47,-227.86,-7.60,13.976586,3.14,2.55,1,1,1,10,39,7,4,0,0,49,3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
2,65.30,23.893333,1.46,1.93,276.71,145.51,67.44,21.403333,5.28,1.61,324.60,334.68,10.08,3.283245,-47.89,-189.17,5.549027,2.4

In [36]:
# save train test split to csv for future use
# separate features from labels
X = PlayFeatures.drop(['trueTargetLabels'], axis = 1)
y = PlayFeatures['trueTargetLabels']

# test train split
X_train, X_test, y_train,  y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

#Train Classifiers

##Logistic Regression

In [ ]:
# initialize grid search parameters
lr_param_grid = param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100],
                              "penalty":['l1', 'l2']}

lr_grid_search = GridSearchCV(LogisticRegression(),
                               param_grid = lr_param_grid,
                               refit = True,
                               cv = 5,
                               verbose = 1,
                               return_train_score = True)

In [ ]:
# cross validate hyperparameters and retrain model
lr_model = lr_grid_search.fit(X_train, y_train)

## Logistic Regression (GAM)

In [ ]:
num_features = X_train.shape[1]

In [ ]:
lams = [np.logspace(-3, 3, 11)] * num_features
num_splines = [5] * num_features

# pygam logistic doesn't work with multinomial target data so use sklearn OvR?
gam_model = OneVsRestClassifier(pygam.LogisticGAM(trueTargetLabel ~ s() + s()), n_jobs=1)\\
            .gridsearch(X_train, y_train,
                        lam = lams,
                        fit_splines = True,
                        return_scores = True,
                        keep_best = True,
                        progress = True)

##RFC


In [44]:
# initialize grid search parameters
rfc_param_grid = {"n_estimators": [100, 200],
                  "max_features": ['sqrt', 'log2'],
                  "max_depth" : [6, 8, 10]}

rfc_grid_search = GridSearchCV(RandomForestClassifier(),
                               param_grid = rfc_param_grid,
                               refit = True,
                               cv = 5,
                               verbose = 3,
                               return_train_score = True)

In [ ]:
# cross validate hyperparameters and retrain model
rfc_model = rfc_grid_search.fit(X_train, y_train)

##Feed Forward Neural Network

In [ ]:
# Define the 3 layer MLP model with batch normalization and dropout
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.bn1 = nn.BatchNorm1d(hidden_dim1)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.bn2 = nn.BatchNorm1d(hidden_dim2)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hidden_dim2, num_classes)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set hyperparameters
input_dim = num_features
hidden_dim1 = 256
hidden_dim2 = 128
num_classes = 5
learning_rate = 0.001
batch_size = 256
num_epochs = 25

# Instantiate the MLP model
model = MLP(input_dim, hidden_dim, num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Step {batch_idx + 1}/{len(train_loader)}, Loss: {total_loss / (batch_idx + 1):.4f}")

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

#Evaluate Classifiers

##Logistic Regression

In [ ]:
# full model test
lr_pred = lr_model.predict(X_test)

print(classification_report(y_test, lr_pred, target_names = ))

cm = confusion_matrix(y_test, lr_pred, labels = )

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = ).plot(xticks_rotation='vertical')
plt.title("")

##GAM

In [ ]:
# full model test
gam_pred = gam_model.predict(X_test)

print(classification_report(y_test, gam_pred, target_names = ))

cm = confusion_matrix(y_test, gam_pred, labels = )

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = ).plot(xticks_rotation='vertical')
plt.title("")

##RFC

In [ ]:
# full model test
rfc_pred = rfc_model.predict(X_test)

print(classification_report(y_test, rfc_pred, target_names = ))

cm = confusion_matrix(y_test, rfc_pred, labels = )

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = ).plot(xticks_rotation='vertical')
plt.title("")

##Feed Forward Neural Network

In [ ]:
# full model test
nn_pred = nn_model.predict(X_test)

print(classification_report(y_test, nn_pred, target_names = ))

cm = confusion_matrix(y_test, nn_pred, labels = )

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = ).plot(xticks_rotation='vertical')
plt.title("")